In [ ]:
NUM_CLASSES = 169

CHANNELS = 1
img_width, img_height = 75, 70

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10
# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 32
BATCH_SIZE_VALIDATION = 32

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import mobilenet

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
 
model_resnet = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(img_width,img_height,3))
model_resnet.layers[-1]

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
for layer in model_resnet.layers:
    layer.trainable = False

# We will have to use the functional API    

# last layers output
x = model_resnet.layers[-1].output
# Flatten as before
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

from tensorflow.keras.models import Model
model = Model(inputs=model_resnet.input, outputs=x)

In [ ]:
from tensorflow.keras import optimizers

sgd = optimizers.SGD(lr = 0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
# sgd = optimizers.Adam(lr = 0.01)

model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# model=Sequential()
# #Still not talking about our train/test data or any pre-processing.

# model = Sequential()

# # 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# # NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
# model.add(model_resnet)

# # 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
# model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

# # Say not to train first layer (ResNet) model as it is already trained
# model.layers[0].trainable = False

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# Batch Normalization helps in faster convergence
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


# flow_From_directory generates batches of augmented data (where augmentation can be color conversion, etc)
# Both train & valid folders must have NUM_CLASSES sub-folders
train_generator = data_generator.flow_from_directory(
        '/content/drive/MyDrive/dataset/train', 
        target_size=(img_width, img_height),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        '/content/drive/MyDrive/dataset/test', 
        target_size=(img_width, img_height),
        batch_size=BATCH_SIZE_VALIDATION,
        class_mode='categorical')

Found 8262 images belonging to 169 classes.
Found 1291 images belonging to 169 classes.


In [ ]:
# Max number of steps that these generator will have opportunity to process their source content
# len(train_generator) should be 'no. of available train images / BATCH_SIZE_TRAINING'
# len(valid_generator) should be 'no. of available train images / BATCH_SIZE_VALIDATION'
(BATCH_SIZE_TRAINING, len(train_generator), BATCH_SIZE_VALIDATION, len(validation_generator))

(32, 259, 32, 41)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)

In [ ]:
fit_history = model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs = 25,
        validation_data=validation_generator,
        validation_steps=10,
        # callbacks=[cb_early_stopper]
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
10/10 [==============================] - 14s 1s/step - loss: 0.7029 - accuracy: 0.8095 - val_loss: 1.6506 - val_accuracy: 0.6562
Epoch 2/25
10/10 [==============================] - 14s 1s/step - loss: 0.6948 - accuracy: 0.8250 - val_loss: 1.6409 - val_accuracy: 0.6344
Epoch 3/25
10/10 [==============================] - 14s 1s/step - loss: 0.6204 - accuracy: 0.8406 - val_loss: 1.7678 - val_accuracy: 0.6281
Epoch 4/25
10/10 [==============================] - 14s 1s/step - loss: 0.6688 - accuracy: 0.8375 - val_loss: 1.6743 - val_accuracy: 0.6438
Epoch 5/25
10/10 [==============================] - 14s 1s/step - loss: 0.6288 - accuracy: 0.8469 - val_loss: 1.4468 - val_accuracy: 0.6969
Epoch 6/25
10/10 [==============================] - 15s 2s/step - loss: 0.7135 - accuracy: 0.8375 - val_loss: 1.8277 - val_accuracy: 0.6125
Epoch 7/25
10/10 [==============================] - 14s 1s/step - loss: 0.6741 - accuracy: 0.8188 - val_loss: 1.8050 - val_accuracy: 0.6031
Epoch 8/25
10/10 [==

In [ ]:
import pickle

In [ ]:
with open('restnet50.pickle','wb') as f:
  pickle.dump(model,f)


INFO:tensorflow:Assets written to: ram://dcadfabc-d1a4-4440-9581-37e3d28cc7b2/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
